# Wege Inland & Zielpersonen

## 1. Import

### 1.1 Bib

In [1]:
#import pandas as pd
import numpy as np
import pandas as pd



### 1.2 Data

In [2]:
wege = pd.read_csv('Data/wege.csv', sep=';', encoding='latin1')
zielpersonen = pd.read_csv('Data/zielpersonen.csv', sep=';', encoding='latin1')



## 2. Bereinigung Daten

In [ ]:
wege.head()
#weg=wege[['HHNR', 'S_X', 'S_Y', 'Z_X', 'Z_Y', 'f51700_weg', 'dauer2', 'wzweck1']]
#weg.rename(columns={'HHNR': 'ID', 'S_X': 'Start_X-Koordinate', 'S_Y': 'Start_Y-Koordinate', 'Z_X': 'Ziel_X-Koordinate', 'Z_Y': 'Ziel_Y-Koordinate', 'f51700_weg': 'Art der Freizeitaktivität', 'dauer2': 'Dauer in Minuten', 'wzweck1': 'Wegzweck'}, inplace=True)
#weg.head()

C:\Users\annhe\AppData\Local\Temp\ipykernel_27372\2424865637.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weg.rename(columns={'HHNR': 'ID', 'S_X': 'Start_X-Koordinate', 'S_Y': 'Start_Y-Koordinate', 'Z_X': 'Ziel_X-Koordinate', 'Z_Y': 'Ziel_Y-Koordinate', 'f51700_weg': 'Art der Freizeitaktivität', 'dauer2': 'Dauer in Minuten', 'wzweck1': 'Wegzweck'}, inplace=True)


,ID,Start_X-Koordinate,Start_Y-Koordinate,Ziel_X-Koordinate,Ziel_Y-Koordinate,Art der Freizeitaktivität,Dauer in Minuten,Wegzweck
0,110001,6.559870,46.549366,6.564175,46.548909,-99,10,5.0
1,110001,6.564175,46.548909,6.573270,46.553510,-99,5,4.0
2,110001,6.573270,46.553510,6.559870,46.549366,-99,10,4.0
3,110001,6.559870,46.549366,6.552598,46.568205,2,10,8.0
4,110001,6.552598,46.568205,6.554009,46.540095,-99,15,5.0


In [ ]:
#länge der Tabelle
len(wege)

151188

In [ ]:
zielpersonen.head()
#zielp=zielpersonen[['HHNR', 'alter', 'gesl', 'f20400a','f20400b', 'f20400c', 'f20400d', 'f40120', 'f40921', 'f81200', 'f50100a', 'f41600_01a', 'f41600_01b', 'f41600_01c', 'f41600_01d', 'f41600_01d', 'f41600_01e', 'f41600_01f', 'f41600_01g','f42100a', 'f42100b', 'f42100c', 'f42100d', 'f42100e', 'zivil', 'ESTATUS', 'T_FREIZEIT']]
#zielp.rename(columns={'HHNR': 'ID', 'alter': 'Alter', 'gesl': 'Geschlecht', 'f20400a': 'Führerausweis PW', 'f20400b': 'Führerausweis Motorräder', 'f20400c': 'Lernfahrausweis PW', 'f20400d': 'Lernfahrausweis Motorräder', 'f40120': 'Ausbildungsniveau', 'f40921': 'Arbeit std./w', 'f81200': 'Arbeitszeitmodelle', 'f50100a': 'Wetter', 'f41600_01a': 'GA-Besitzer', 'f41600_01b': 'Halbtax-Besitzer', 'f41600_01c': 'Verbund-Abo-Besitzer', 'f41600_01d': 'Streckenabo-Besitzer', 'f41600_01e': 'seven25-Besitzer', 'f41600_01f': 'Juniortkarten-Besitzer', 'f41600_01g': 'Andere Abos', 'f42100a': 'Velo-Besitzer', 'f42100b': 'Mofa-Besitzer', 'f42100c': 'Kleinmotorrad-Besitzer', 'f42100d': 'Motorrad-Besitzer', 'f42100e': 'Auto-Besitzer', 'zivil': 'Familienstand', 'ESTATUS': 'Erwerbsstatus', 'T_FREIZEIT': 'Täfliche Freizeitwege'}, inplace=True)
#zielp.head()

,HHNR,alter,gesl,f20400a,f20400b,f20400c,f20400d,f40120,f40921,f81200,...,f41600_01f,f41600_01g,f42100a,f42100b,f42100c,f42100d,f42100e,zivil,ESTATUS,T_FREIZEIT
0,110001,57,1,1,1,-99,-99,4,-99,-99,...,2,2,3,3,3,1,1,4,3,3
1,110008,57,1,1,1,-99,-99,2,-99,-99,...,2,2,3,3,3,1,3,2,5,2
2,110011,16,2,-99,-99,-99,-99,2,-99,-99,...,2,2,1,3,3,-99,-99,1,4,1
3,110024,17,2,-99,-99,2,2,2,-99,-99,...,2,2,1,2,2,-99,-99,1,4,0
4,110026,53,2,1,1,-99,-99,4,-99,-99,...,2,2,1,3,3,1,2,1,8,3


In [11]:
len(zielpersonen)

55018

Die Datensätze beinhalten keine Null-Werte und keine Duplikate

## 3. Merge

In [12]:
# Right Join der DataFrames auf Basis der Spalte 'HHNR'
merged_data = wege.merge(zielpersonen, on='HHNR', how='left')


In [13]:
# Finde alle Wege, denen keine Zielperson zugeordnet ist
no_zielperson = merged_data[merged_data['alter'].isna()]  # 'alter' ist ein Beispiel; ersetze es durch eine Spalte, die nur in 'zielpersonen' existiert

# Gib die HHNR-Werte aus, die keine Zielperson zugeordnet haben
nicht_zugeordnete_wege = no_zielperson['HHNR'].unique()  # Gibt die eindeutigen HHNR ohne Zuordnung

print("Anzahl der Wege ohne zugeordnete Zielperson:", len(nicht_zugeordnete_wege))
print("HHNR-Werte ohne zugeordnete Zielperson:", nicht_zugeordnete_wege)

Anzahl der Wege ohne zugeordnete Zielperson: 0
HHNR-Werte ohne zugeordnete Zielperson: []


In [14]:
len(merged_data)

151188

In [15]:
merged_data.head()

,HHNR,WP_x,AUSNR,WEGNR,DMOD_x,f51100,f51400,f51100time,f51400time,pseudo,...,AU_KANTON,AU_BFS,AU_PLZ,AU_AGGLO2012,AU_AGGLO_GROESSE2012,AU_staedt_char_2012,AU_stat_stadt_2012,AU_stadt_land_2012,AU_DEGURBA,AU_OeV_KLASSE
0,110001,1.606924,1,1,2,555,565,9:15:00,9:25:00,1,...,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99
1,110001,1.606924,1,2,2,575,580,9:35:00,9:40:00,1,...,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99
2,110001,1.606924,1,3,2,610,620,10:10:00,10:20:00,1,...,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99
3,110001,1.606924,2,4,2,750,760,12:30:00,12:40:00,1,...,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99
4,110001,1.606924,2,5,2,810,825,13:30:00,13:45:00,1,...,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99


In [18]:
merged_data.columns

Index(['HHNR', 'WP_x', 'AUSNR', 'WEGNR', 'DMOD_x', 'f51100', 'f51400',
       'f51100time', 'f51400time', 'pseudo',
       ...
       'AU_KANTON', 'AU_BFS', 'AU_PLZ', 'AU_AGGLO2012', 'AU_AGGLO_GROESSE2012',
       'AU_staedt_char_2012', 'AU_stat_stadt_2012', 'AU_stadt_land_2012',
       'AU_DEGURBA', 'AU_OeV_KLASSE'],
      dtype='object', length=238)

In [20]:
#gib mit unique WErte aus der Spalte f51700w aus
merged_data['f51700_weg'].unique()

array([-99,   2,  16,   7,   3,   1,   4,  10,  15,   9,  90,  13,  12,
        14,   5,   6,  17,   8,  11,  22, -98, -97])

Notizen

wzweck1 Filter 8 Für Freizeitwege 

f51700w ist welche Freizeitaktivität wurde gemacht
Aktivitäten sind katagorisiert / Alle nichtaussagen zusammennehmen 
Kategorien









